<a href="https://colab.research.google.com/github/Richards-R/FCC-Book-Recommendation-Engine-using-KNN/blob/main/Richards_R_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-12-29 20:03:49--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  76.5MB/s    in 0.3s    

2023-12-29 20:03:49 (76.5 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books.loc[df_books['author'] =="Anne Rice"]

,isbn,title,author
609,0345369947,The Mummy or Ramses the Damned,Anne Rice
979,3442098424,Der FÃ?Â¼rst der Finsternis. Roman.,Anne Rice
1133,0345422392,Vittorio the Vampire: New Tales of the Vampires,Anne Rice
1656,0345367898,Witching Hour (Lives of the Mayfair Witches),Anne Rice
1764,0375411992,Blackwood Farm (The Vampire Chronicles),Anne Rice
...,...,...,...
264289,0517357682,Servant of the Bones,Anne Rice
265425,0375702180,Pandora: New Tales of the Vampires (Random Hou...,Anne Rice
266224,0394579453,The Mummy or Ramses the Damned,Anne Rice
268846,2266044591,LA Reine Des Damnes,Anne Rice


In [ ]:
df_ratings.sample(10)

,user,isbn,rating
1095587,262998,0316038121,0.0
492814,117852,0099282194,0.0
189318,41700,3802529138,0.0
621359,150498,0451085108,0.0
814974,198408,2743611456,0.0
951755,230522,0345362985,0.0
804747,195694,0451455150,0.0
971538,234623,0736413065,0.0
886050,213350,0674306880,0.0
842575,203968,0812507371,0.0


In [ ]:
df_ratings.shape

(1149780, 3)

In [ ]:
df_ratings.describe()

,user,rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [ ]:
df_ratings.loc[df_ratings['user']==11676]

,user,isbn,rating
45326,11676,9022906116,7.0
45327,11676,"\0432534220\""""",6.0
45328,11676,"\2842053052\""""",7.0
45329,11676,0 7336 1053 6,0.0
45330,11676,0=965044153,7.0
...,...,...,...
58923,11676,O77O428452,0.0
58924,11676,O805063196,0.0
58925,11676,O9088446X,8.0
58926,11676,X000000000,10.0


In [ ]:
df_merged = df_ratings.merge(df_books, on='isbn', how='left')

In [ ]:
df_merged.head(3)

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5.0,Rites of Passage,Judith Rae
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks


In [ ]:
df_merged.shape

(1149780, 5)

In [ ]:
df_merged.sample(5)

,user,isbn,rating,title,author
36182,8681,0312242727,0.0,A Writer's Workbook : Daily Exercises for the ...,Caroline Sharp
523669,126903,0446600342,0.0,Where There's Smoke,Sandra Brown
844171,204507,0023885815,8.0,Introduction to the Profession of Counseling,Frank A. Nugent
891316,215071,0375411968,5.0,Burning Marguerite,ELIZABETH INNESS-BROWN
1119605,268622,1561003913,0.0,Politically Correct Bedtime Stories,James Pinn Garner


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149780 entries, 0 to 1149779
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
 3   title   1031175 non-null  object 
 4   author  1031174 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 43.9+ MB


In [ ]:
#Clean code
#Find uers with lessthan 200 reviews
#Find books with less than 100 reviews
users = df_merged['user'].value_counts()
books = df_merged['isbn'].value_counts()

users = users[users >= 200].index
books = books[books >= 100].index


In [ ]:
print(len(users.values))
print(len(books.values))

905
731


In [ ]:
#Make a new data frame
df = df_merged.loc[(df_merged['user'].isin(users.values)) & (df_merged['isbn'].isin(books.values))]

In [ ]:
df.describe()

,user,rating
count,49781.000000,49781.000000
mean,140993.388562,2.024086
std,80901.415891,3.576591
min,254.000000,0.000000
25%,70415.000000,0.000000
50%,140358.000000,0.000000
75%,213350.000000,3.000000
max,278418.000000,10.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49781 entries, 1456 to 1147441
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49781 non-null  int32  
 1   isbn    49781 non-null  object 
 2   rating  49781 non-null  float32
 3   title   49517 non-null  object 
 4   author  49517 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.9+ MB


In [ ]:
print(users, books)

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
             28634,  59727, 268622, 188951, 225595,  83671, 252827,  99955,
             36554,  26883],
           dtype='int64', length=905) Index(['0971880107', '0316666343', '0385504209', '0060928336', '0312195516',
       '044023722X', '0679781587', '0142001740', '067976402X', '0671027360',
       ...
       '0060916508', '067100042X', '0425189864', '0385424736', '0140092323',
       '0786866586', '0449203794', '0425151867', '0312966806', '0425136981'],
      dtype='object', length=731)


In [ ]:
df = df.drop_duplicates(subset=['title', 'user'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49374 entries, 1456 to 1147441
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49374 non-null  int32  
 1   isbn    49374 non-null  object 
 2   rating  49374 non-null  float32
 3   title   49136 non-null  object 
 4   author  49136 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.9+ MB


In [ ]:
df.isna().sum()

user        0
isbn        0
rating      0
title     238
author    238
dtype: int64

In [ ]:
df = df.dropna(how='any')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49136 entries, 1456 to 1147441
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49136 non-null  int32  
 1   isbn    49136 non-null  object 
 2   rating  49136 non-null  float32
 3   title   49136 non-null  object 
 4   author  49136 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.9+ MB


In [ ]:
df_pivot = df.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
df_pivot.head(5)

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_matrix = csr_matrix(df_pivot.values)
print(df_matrix)

  (0, 0)	9.0
  (0, 36)	9.0
  (0, 41)	8.0
  (0, 58)	7.0
  (0, 136)	9.0
  (0, 157)	10.0
  (0, 171)	9.0
  (0, 188)	8.0
  (0, 220)	10.0
  (0, 221)	10.0
  (0, 246)	10.0
  (0, 278)	10.0
  (0, 391)	10.0
  (0, 477)	9.0
  (0, 507)	9.0
  (0, 554)	10.0
  (0, 602)	9.0
  (0, 693)	9.0
  (0, 761)	8.0
  (0, 768)	7.0
  (0, 827)	10.0
  (0, 876)	10.0
  (1, 10)	9.0
  (1, 25)	8.0
  (1, 41)	9.0
  :	:
  (671, 232)	8.0
  (671, 271)	6.0
  (671, 330)	9.0
  (671, 437)	9.0
  (671, 475)	10.0
  (671, 647)	8.0
  (671, 660)	7.0
  (671, 746)	7.0
  (671, 759)	9.0
  (672, 15)	8.0
  (672, 25)	7.0
  (672, 112)	5.0
  (672, 151)	6.0
  (672, 234)	7.0
  (672, 292)	8.0
  (672, 305)	9.0
  (672, 324)	10.0
  (672, 369)	10.0
  (672, 386)	9.0
  (672, 498)	10.0
  (672, 505)	8.0
  (672, 591)	8.0
  (672, 712)	8.0
  (672, 830)	9.0
  (672, 880)	8.0


In [ ]:
nn = NearestNeighbors(metric='cosine')
nn.fit(df_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  #Create a list to put our model outputs into
  recommended_books = [book,[]]

  #Pass through our book
  distance, book_info = nn.kneighbors([df_pivot.loc[book]], 6, return_distance=True)

  #Gather the text and distance & reverse the order
  recom_book_info = df_pivot.iloc[np.flip(book_info[0])[:-1]].index.to_list()
  recom_distance = list(np.flip(distance[0])[:-1])

  #For each value in our two variables append to our empty book list above
  for r in zip(recom_book_info, recom_distance):
    recommended_books[1].append(list(r))

  return recommended_books

In [ ]:
df_books.head(3)


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este


In [ ]:
get_recommends('4 Blondes')

['4 Blondes',
 [['On the Road', 0.84285647],
  ['Bridget Jones: The Edge of Reason', 0.8379612],
  ['The Corrections', 0.8377818],
  ['Seabiscuit', 0.83020264],
  ['The House of the Spirits', 0.79863167]]]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
